### 1. Setting up the Environment

In [1]:
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import base64
import pickle
import openai
import requests

### 2. Load the env

In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

### 3. Gmail Authentication

In [5]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def get_gmail_service():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
            
        with open('token.json', 'wb') as token:
            token.write(creds.to_json())
            
    service = build('gmail', 'v1', credentials=creds)
    print("Gmail service created successfully.")
    return service

### 4. Fetch unread email IDs

In [6]:
def get_unread_emails(service, max_results=5):
    results = service.users().messages().list(
        userId='me', 
        labelIds=['INBOX'], 
        q='is:unread', 
        maxResults=max_results
        ).execute()
    return results.get('messages', [])


### 5. Extract email content

In [7]:
def get_email_body(service, msg_id):
    msg = service.users().messages().get(userId='me', id=msg_id, format='full').execute()
    parts = msg['payload'].get('parts', [])
    for part in parts:
        if part['mimeType'] == 'text/plain':
            data = part['body']['data']
            text = base64.urlsafe_b64decode(data).decode('utf-8')
            return text
    return "No readable content found."


### 6. Summarize with GPT

In [ ]:
def summarize_with_gpt(text):
    prompt = f"Summarize this email:\n\n{text}"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "Content": "You are a professional assistant. Summarize emails concisely and formally."},
                  {"role": "user", "content": prompt}
                  ],
        max_tokens=150
    )
    summary =  response.choices[0].message.content.strip()
    return summary


### 7. Send to Telegram